### Import cw-simulate module and create new instance

In [ ]:
import { SimulateCosmWasmClient } from "@oraichain/cw-simulate";

senderAddress = "orai14vcw5qk0tdvknpa38wz46js5g7vrvut8lk0lk6";
client = new SimulateCosmWasmClient({
  chainId: "Oraichain",
  bech32Prefix: "orai",
  metering: true
});


### Setup account balance

In [ ]:
import { coins } from '@cosmjs/stargate';

client.app.bank.setBalance(senderAddress, coins('10000000', 'orai'));

console.log(await client.getBalance(senderAddress, 'orai'));

### Deploy contract on SimulateCosmwasmClient

In [ ]:
import { Cw20BaseClient } from "@oraichain/common-contracts-sdk";

const wasmCode = new Uint8Array(await fetch("/wasm/cw20-base.wasm").then((res) => res.arrayBuffer()));
const { codeId } = await client.upload(senderAddress, wasmCode, "auto");
const { contractAddress } = await client.instantiate(
  senderAddress,
  codeId,
  {
    decimals: 6,
    symbol: "CVT",
    name: "cosmoverse token",
    mint: { minter: senderAddress },
    initial_balances: [{ address: senderAddress, amount: "1000000000" }]
  },
  "cw20 token",
  "auto"
);

cw20Contract = new Cw20BaseClient(client, senderAddress, contractAddress);

console.log(await cw20Contract.tokenInfo());

### Console.log in smart contract

1. run `code ../cw-plus/contracts/cw20-base` to add log function
1. run `cwtools build ../cw-plus/contracts/cw20-base -d -w -o public/wasm/` to override wasm contract file
1. re-run notebook above to see the log


### Execute contract with gas metering

In [ ]:
const { gasUsed } = await cw20Contract.transfer({amount: '1000000', recipient: 'orai1ur2vsjrjarygawpdwtqteaazfchvw4fg6uql76'});
console.log(`Transfer token required gas: ${gasUsed}`);

console.log(await cw20Contract.balance({address:'orai1ur2vsjrjarygawpdwtqteaazfchvw4fg6uql76'}));

### Load smart contract with state from running blockchain

In [ ]:
import { BufferCollection, SortedMap, compare } from "@oraichain/cw-simulate";
import { Cw20BaseClient } from "@oraichain/common-contracts-sdk";

const wasmCode = new Uint8Array(await fetch("/wasm/kwt-token.wasm").then((res) => res.arrayBuffer()));
const { codeId } = await client.upload(senderAddress, wasmCode, "auto");

const buffer = new Uint8Array(await fetch("/wasm/kwt-token.state").then((res) => res.arrayBuffer()));
const state = SortedMap.rawPack(new BufferCollection(buffer), compare);

const kwtContract = new Cw20BaseClient(client, senderAddress, "orai1nd4r053e3kgedgld2ymen8l9yrw8xpjyaal7j5");
await client.loadContract(
  "orai1nd4r053e3kgedgld2ymen8l9yrw8xpjyaal7j5",
  {
    codeId,
    admin: senderAddress,
    label: "kwt token",
    creator: senderAddress,
    created: 1
  },
  state
);

console.log(await kwtContract.allAccounts({ limit: 5 }));


### Cosmwasm with Zero Knowledge extension

#### ZK Smart contract code

```rust
#[cfg(not(feature = "library"))]
use cosmwasm_std::entry_point;
use cosmwasm_std::{to_binary, Binary, Deps, DepsMut, Env, MessageInfo, Response, StdResult};
use crate::msg::{ExecuteMsg, InstantiateMsg, QueryMsg};
use crate::state::CURVE;


#[cfg_attr(not(feature = "library"), entry_point)]
pub fn instantiate(
    deps: DepsMut,
    _env: Env,
    info: MessageInfo,
    msg: InstantiateMsg,
) -> Result<Response, ContractError> {    
    CURVE.save(deps.storage, &msg.curve)?;
    Ok(Response::default())
}

#[cfg_attr(not(feature = "library"), entry_point)]
pub fn execute(
    deps: DepsMut,
    _env: Env,
    info: MessageInfo,
    msg: ExecuteMsg,
) -> Result<Response, ContractError> {
    match msg {
        ExecuteMsg::SetVKRaw {
            vk_raw
        } => {            
            VK.save(deps.storage, &vk_raw)?;
        }        
    }
}


#[cfg_attr(not(feature = "library"), entry_point)]
pub fn query(deps: Deps, _env: Env, msg: QueryMsg) -> StdResult<Binary> {
    match msg {
        QueryMsg::VerifyProofJson {
            proof_raw,
            public_inputs,            
        } => to_binary(&query::verify_proof(
            deps,            
            &proof_raw,
            &public_inputs,
        )?),
    }
}

pub fn verify_proof(
    deps: Deps,    
    proof_raw: &[u8],
    public_inputs: &[u8],
) -> StdResult<bool> {
    let vk = VK.load(deps.storage)?;    
    let curve = CURVE.load(deps.storage)?;
    let verified = deps
            .api
            .groth16_verify(&public_inputs, proof_raw, &vk, curve)
    Ok(verified)
}

```

### Change Backend Api with ZK extensions

In [ ]:
import { BasicBackendApi } from "@oraichain/cw-simulate";
import { groth16_verify } from "@oraichain/cosmwasm-vm-zk-web";

// load wasm libraries
await require("@oraichain/cosmwasm-vm-zk-web").default();

// extends vm host functions
class ZkBackendApi extends BasicBackendApi {  
  groth16_verify(input, proof, vk, curve) {
    return groth16_verify(input, proof, vk, curve);
  }
}

client.app.backendApi = new ZkBackendApi('orai');

### Deploy ZK contract on extended SimulateCosmwasmClient

In [ ]:
const wasmCode = new Uint8Array(await fetch("/wasm/zk.wasm").then((res) => res.arrayBuffer()));
const { codeId } = await client.upload(senderAddress, wasmCode, "auto");
contractAddress = (
  await client.instantiate(
    senderAddress,
    codeId,
    {
      curve: "bn254"
    },
    "groth16 verifier",
    "auto"
  )
).contractAddress;

### Execute contract with gas metering

In [ ]:
PUBLIC_INPUTS =
  't6CNWWLx3Fd4+fI4XCkc5vdvmwdeAo5lAPMnIDvrxh9SGQJVQa0SBUqvbA7oxa4J8jtpMGipzID9lg9mbNZRIeD7lRlesjw8ZdffRKet+Dhx7AOAGJ0+dXQXdl1Rrg0q';
PROOF =
  'Ig2y4hzjpMsVvHC96ppAv68XvyNrigWimFBtG3/ixK62J5Wk3EEMx2j7zwlWFV5KcftnhcaRTTtuqd5fp7SZJZH/uvmMWEdM0GKrmoE/oFoXrvh0eaTlxNjoteRLDQGGkkHa7zjdUgdKBndWTokOBXYaw2xsn/I9g1a5rW2a6y8=';
VK =
  'qNKepAYpvnYvKhK9p8xFuZijTEOpbExnRMjHqQDo+Ape7Ob6V3FInLAwb0ma2Roz0BWfKXhjMteC24cKCYBECqEiWtI8DkdsfTa3luaptQJAhBtL6VXRPqVN2NoBEo4M+SCUtFn/iCeAq98+F4TAbfbIXAAG/X8ll+PpBS2SFSdPCPxPlNyBKfKaV43Bf16mDqhdLIingpS3ktvy+o0wlzuAymtWdGO2kLizqPcOtkaCJzWOXzFuuBUKkhUrdTUZxMqCf+wX9yg9FSKHZ7Vrc27JSY85/ltRGor1A7Y6GZcEAAAAAAAAALnIa74+XvNJDV20eL4KeTOTTktaFI4sAWArR1yD4lAIkdxEpv4vMx2ptm81YjmKdiZ397fJXTqCqWmODPYhISjW6yej8Rq7UqmKUJvxUC8JR+mrnB1yoIYUDA0xaGbWJMIqafjftZV+NdjT01CzyD7pXoiXx7dtQYgWg9JWHNkZ';
  
const res = await client.execute(senderAddress, contractAddress, {
    set_vk_raw: {
      vk_raw: VK,
    },
});

console.log(res);

### Verify ZK Proof

In [ ]:
const res = await client.queryContractSmart(contractAddress, {
    verify_proof_json: {
      proof_raw: PROOF,
      public_inputs: PUBLIC_INPUTS,
    },
});
  
console.log(res);